#  More Quasi-Newton Methods

In this discussion, we will talk about:
* The SR1 method
* DFP method
* BFGS method

---

## Quasi-Newton Methods

Recall that Newton's method is defined by the iteration
$$ \mathbf{x}_{k+1} = \mathbf{x}_k - \left[\nabla^2 f_k\right]^{-1}\nabla f_k $$

which can be viewed as a line search algorithm with $\mathbf{p}_k = -\left[\nabla^2 f_k\right]^{-1}\nabla f_k$, called the Newton direction, and $\alpha_k=1$.

We showed several nice theoretical results for Newton's method, including that it converges in *a single step* for quadratic functions and that it converges quadratically for most others, provided our initial guess is "close enough" to the minimizer. Despite these ostensibly great theoretical results, there are several drawbacks to Newton's method, the most pressing being the calculation and then inversion of the Hessian matrix, requiring a lot of computation during each iteration, especially for very high-dimensional functions.

In an attempt to remedy these issues, we introduced a family of **quasi-Newton methods**, which approximate the Hessian with some positive definite matrix $B_k$ during each iteration, or even better, approximate the inverse of the Hessian directly with a matrix $H_k$. One iteration of a quasi-Newton method takes the following form:
$$\mathbf{x}_{k+1} = \mathbf{x}_k - \alpha_kB_k^{-1}\nabla f_k \equiv \mathbf{x}_k - \alpha_kH_k\nabla f_k$$

where $B_k$ is a positive definite approximation of the Hessian $\nabla^2 f_k$ and/or $H_k$ is a positive definite approximation of the inverse Hessian $[\nabla^2f_k]^{-1}$.

Since the actual Hessian satisfies
$$ \nabla^2f_k(\mathbf{x}_{k+1}-\mathbf{x}_k)\approx \nabla f_{k+1}-\nabla f_k $$

we choose $B_k$ such that it has the same property, that is determine a solution to

$$ B_{k+1}(\mathbf{x}_{k+1}-\mathbf{x}_k)= \nabla f_{k+1}-\nabla f_k,\qquad\text{or}\qquad B_{k+1}\mathbf{s}_k = \mathbf{y}_k $$

with $\mathbf{s}_k\equiv \mathbf{x}_{k+1}-\mathbf{x}_k$ and $\mathbf{y}_k\equiv\nabla f_{k+1}-\nabla f_k$. The above, called the **secant equation**, is equivalent to the following requirements for the inverse approximation, $H_{k+1}$, sometimes called the **dual secant equation**:

$$ H_{k+1}(\nabla f_{k+1}-\nabla f_k)=\mathbf{x}_{k+1}-\mathbf{x}_k,\qquad\text{or}\qquad H_{k+1}\mathbf{y}_k =\mathbf{s}_k $$


> **Note:** Observe the peculiarity of the (dual) secant equation, which is not a typical system of equations. Indeed, here we seek a *matrix* satisfying a certain condition, not a vector. There are only $n$ equations determining the $n^2$ components of an $n\times n$ matrix, so the system is undertermined and thus admits infinitely many solutions. Even with the requirement that $H_{k+1}$ must be SPD, we are still left with $n(n-1)/2$ degrees of freedom. We can thus impose various other requirements on $H_{k+1}$, each choice giving rise to a different quasi-Newton method. The key feature of all quasi-Newton methods is that they do not recalculate $H_{k+1}$ from scratch during each iteration; instead they rely on some recursive definition, determining $H_{k+1}$ by using some low-rank update to the old approimation $H_k$.

### Symmetric Rank-1 (SR1) Method

The simplest quasi-Newton method is the **symmetric rank-1 (SR1) method** which requires updates to the inverse Hessian to be of the form

$$ H_{k+1} = H_k + \mathbf{z}_k\mathbf{z}_k^T $$

where $\mathbf{z}_k$ is a vector chosen to satisfy the secant equation, and $H_0$ can be chosen as any positive definite initial approximation (often simply $H_0\equiv I$). The rank one update is then determined by the secant equation to be

$$ \mathbf{z}_k\mathbf{z}_k^T = \frac{(\mathbf{s}_k - H_k \mathbf{y}_k) (\mathbf{s}_k - H_k \mathbf{y}_k)^T}{(\mathbf{s}_k - H_k \mathbf{y}_k)^T \mathbf{y}_k} $$

This update scheme is beneficial in that no explicit calculation or inversion of the Hessian is needed, just matrix multiplication at each step. Further, each update is guaranteed to be symmetric due to the form of the rank 1 update. However, a rather concerning issue with SR1 is that the update does not necessarily preserve positive definiteness of the inverse Hessian. Indeed, examining the denominator,

$$ (\mathbf{s}_k-H_k\mathbf{y}_k)^T\mathbf{y}_k = \mathbf{s}_k^T\mathbf{y}_k-\mathbf{y}_k^TH_k^T\mathbf{y}_k $$

the first term is positive if $H_{k+1}$ is positive definite, which can be seen by premultiplying the secant equation with $\mathbf{y}_k^T$. But if the old update $H_k$ is also positive definite, the second term may be larger than the first term, making the update matrix $\mathbf{z}_k\mathbf{z}_k^T$ negative definite (the denominator is its only eigenvalue), decreasing the eigenvalues of $H_{k+1}$ relative to $H_k$, and possibly pushing them negative. If this ever happens, we are not guaranteed descent in general.

While there are ways around this (e.g. some modified version of the LM algorithm), you may have spotted an even more fatal issue, specifically that the denominator may even *vanish* if the two vectors happen to be orthogonal or very close to it, causing the update not to even be defined! This can be interpreted as an artifact of requiring a rank-1 update as a solution to the secant equation, which may not exist in general.

In practice, this happens rarely, but in case it does, the typical workaround is to simply skip updating that iteration, i.e. set $H_{k+1}=H_k$, if the magnitude of the denominator is smaller than some arbitrarily chosen constant, e.g. don't udpate if $|(\mathbf{s}_k-H_k\mathbf{y}_k)^T\mathbf{y}_k|<10^{-8}\|\mathbf{s}_k-H_k\mathbf{y}_k\|\|\mathbf{y}_k\|$. The hope is that in future iterations $\mathbf{s}_k$ and $\mathbf{y}_k$ are modified such that the problem is avoided. Empirically this does not appear to have very much effect on the convergence of the iteration.

All together, then, the SR1 method can be described as follows:

1. Set $H_0=I$, the identity matrix.
2. Update $\mathbf{x}_{k+1} = \mathbf{x}_k - \alpha_k H_k\nabla f_k$, where $\alpha_k$ is chosen using any step size method, e.g. exact line search, backtracking, etc.
3. Define $\mathbf{s}_k \equiv \mathbf{x}_{k+1}-\mathbf{x}_k$, $\mathbf{y}_k\equiv\nabla f_{k+1}-\nabla f_k$, and check if $|(\mathbf{s}_k-H_k\mathbf{y}_k)^T\mathbf{y}_k|<10^{-8}\|\mathbf{s}_k-H_k\mathbf{y}_k\|\|\mathbf{y}_k\|$.
4. If true, set $H_{k+1}=H_k$ (skipping the update). If false, set $H_{k+1} = H_k + \dfrac{(\mathbf{s}_k - H_k \mathbf{y}_k) (\mathbf{s}_k - H_k \mathbf{y}_k)^T}{(\mathbf{s}_k - H_k \mathbf{y}_k)^T \mathbf{y}_k}$.
5. Repeat 2-4 until convergence.

Despite the several drawbacks mentioned above, under ideal conditions, convergence is typically *superlinear* at worst, $p>1$. (Contrast with "pure" Newton, which is quadratic, $p=2$, at worst.) Indeed we showed in the lecture that SR1 converges *in a finite number of steps* (at most $n$ if $\mathbf{x}\in\mathbb{R}^n$) if $f$ is a quadratic function and $\alpha_k$ is chosen through exact line search. (Contrast with pure Newton, which converges in a *single* step.)

### Davidon-Fletcher-Powell (DFP) method

Another quasi-Newton method was actually proposed before SR1 and is called the **Davidon-Fletcher-Powell (DFP) method** after its inventors. Rather than requiring a rank 1 update to the inverse Hessian, DFP focuses on the Hessian approximation itself, $B_{k+1}$. Since the secant equation underdetermines this matrix, the DFP update is constrained such that $B_{k+1}$ is the "closest" matrix to $B_k$ in the following sense:

$$ B_{k+1} = \min_B \|B-B_k\|_{F,W}\qquad \text{subject to } B=B^T, B\mathbf{s}_k=\mathbf{y}_k $$

Here the norm is chosen to be the *weighted [Frobenius norm](https://mathworld.wolfram.com/FrobeniusNorm.html)*, $\|A\|_{F,W} = \|W^{1/2}AW^{1/2}\|_F$, and $\|C\|_F^2 = \sum_{i,j} c_{ij}^2$. The weight matrix $W$ need only satisfy $W\mathbf{s}_k=\mathbf{y}_k$. The reasoning behind this choice of norm is that it makes the minimization problem relatively straightforward to solve as a least squares problem, the solution being

$$ B_{k+1} = \left(I-\frac{\mathbf{y}_k\mathbf{s}_k^T}{\mathbf{y}_k^T\mathbf{s}_k}\right)B_k\left(I-\frac{\mathbf{s}_k\mathbf{y}_k^T}{\mathbf{y}_k^T\mathbf{s}_k}\right) + \frac{\mathbf{y}_k\mathbf{y}_k^T}{\mathbf{y}_k^T\mathbf{s}_k} $$

For a complete derivation, see [this StackExchange answer](https://math.stackexchange.com/questions/2091867/quasi-newton-methods-understanding-dfp-updating-formula).

In practice, this calculation is never done because we don't really care about $B_{k+1}$, only its inverse, $H_{k+1}$. To obtain the inverse, we make use of the following useful theorem, the result of which is called the **Sherman-Morrison formula**:

If $\bar{A}=A+\mathbf{u}\mathbf{v}^T$ is obtained by a rank-1 update, the inverse,

$$ \bar{A}^{-1} = A^{-1} - \frac{A^{-1}\mathbf{u}\mathbf{v}^TA^{-1}}{1+\mathbf{v}^TA^{-1}\mathbf{u}} $$

provided the denominator does not vanish. Applying this to the above leads to the rather nice result,

$$ H_{k+1} = B_{k+1}^{-1} = H_k + \frac{\mathbf{s}_k \mathbf{s}_k^T}{\mathbf{y}_k^{T} \mathbf{s}_k} - \frac{H_k \mathbf{y}_k \mathbf{y}_k^T H_k}{\mathbf{y}_k^T H_k \mathbf{y}_k} $$

which can immediately be recognized as a *rank two* update to the old approximation $H_k$.

Furthermore, it is clear that the denominators of each fraction 1) never vanish unless $\mathbf{y}_k=\mathbf{0}\implies \nabla f_{k+1}=\nabla f_k$, i.e. convergence, and 2) are always nonnegative. In fact, it is also proven in lecture that the DFP update not only preserves the symmetry of the inverse Hessian, it also preserves the positive definiteness! These facts make it a much more reliable method than SR1, at least analytically, in that there are no ad hoc skipped steps, and descent is always guaranteed.

However, in practice, it turns out the SR1 method's approximations to the Hessian tend to be much closer to the true value than DFP's approximations. Furthermore, DFP tends to "get stuck" due to the fact that the inverse Hessian can become nearly singular, meaning each update does not progress very far, even with an optimal step size.

### Broyden-Fletcher-Goldfarb–Shanno (BFGS) method

The final quasi-Newton method we will introduce is the most popular for a variety of reasons. It is called the **Broyden-Fletcher-Goldfarb–Shanno (BFGS) method** and is very similar to the DFP method. Indeed the exact same line of reasoning can be applied as in DFP – require that the update be "close" to the old value in some norm – except the argument is applied to the inverse Hessian $H_{k+1}$ rather than the Hessian $B_{k+1}$ itself. In fact, the form of the BFGS update to the inverse Hessian looks extremely similar to the DFP update for the regular Hessian:

$$ H_{k+1} = \left(I-\frac{\mathbf{s}_k\mathbf{y}_k^T}{\mathbf{y}_k^T\mathbf{s}_k}\right)H_k\left(I-\frac{\mathbf{y}_k\mathbf{s}_k^T}{\mathbf{y}_k^T\mathbf{s}_k}\right) + \frac{\mathbf{s}_k\mathbf{s}_k^T}{\mathbf{y}_k^T\mathbf{s}_k} $$

Indeed, this is *exactly* the same form as the DFP update to the Hessian, with the substitutions $H\leftrightarrow B$, $\mathbf{y}\leftrightarrow\mathbf{s}$. The same translates over to the (rarely used in practice) calculation of the BFGS update to the Hessian, which takes the same form as the DFP update to the inverse Hessian:

$$ B_{k+1} = B_k + \frac{\mathbf{y}_k \mathbf{y}_k^T}{\mathbf{y}_k^{T} \mathbf{s}_k} - \frac{B_k \mathbf{s}_k \mathbf{s}_k^T B_k}{\mathbf{s}_k^T B_k \mathbf{s}_k} $$

In practice, the above form of $H_{k+1}$ is not what is used for calculation due to requiring the construction of multiple temporary matrices (each bracketed term). Instead, we can expand brackets, noting that $\mathbf{y}_k^TH_k\mathbf{y}_k$ is a scalar, to get

$$ \begin{align*}
    H_{k+1} &= \left(I-\frac{\mathbf{s}_k\mathbf{y}_k^T}{\mathbf{y}_k^T\mathbf{s}_k}\right)\left(H_k-\frac{H_k\mathbf{y}_k\mathbf{s}_k^T}{\mathbf{y}_k^T\mathbf{s}_k}\right) + \frac{\mathbf{s}_k\mathbf{s}_k^T}{\mathbf{y}_k^T\mathbf{s}_k} \\
    &= H_k - \frac{H_k\mathbf{y}\mathbf{s}_k^T}{\mathbf{y}_k^T\mathbf{s}_k} - \frac{\mathbf{s}_k\mathbf{y}_k^TH_k}{\mathbf{y}_k^T\mathbf{s}_k} + \frac{\mathbf{s}_k\mathbf{y}_k^TH_k\mathbf{y}_k\mathbf{s}_k^T}{(\mathbf{y}_k^T\mathbf{s}_k)^2} + \frac{\mathbf{s}_k\mathbf{s}_k^T}{\mathbf{y}_k^T\mathbf{s}_k} \\
    &= H_k + (\mathbf{y}_k^T\mathbf{s}_k+\mathbf{y}_k^T H_k \mathbf{y}_k)\frac{\mathbf{s}_k \mathbf{s}_k^T}{(\mathbf{y}_k^T \mathbf{s}_k)^2} - \frac{H_k \mathbf{y}_k \mathbf{s}_k^T + \mathbf{s}_k \mathbf{y}_k^TH_k}{\mathbf{y}_k^T\mathbf{s}_k} 
\end{align*}$$

Empirically, BFGS does not suffer from the same issues as DFP. It tends to be much more efficient and the inverse Hessian does not tend to become nearly singular as quickly as in DFP.

On Thursday, we will implement each of these methods on a single function and compare the convergence results.